# x402 facilitor for Optimism 

In this notebook, we test the x402 facilitator for Optimism network.

In [14]:
import requests
import json
from datetime import datetime

# Facilitator endpoint (local testing or deployed)
FACILITATOR_URL = "http://localhost:8080"  # Change to deployed URL when available

In [16]:
from web3 import Web3
from eth_account.messages import encode_typed_data
from eth_account import Account
import os
from dotenv import load_dotenv
import secrets

# Load environment variables
load_dotenv()
private_key = os.getenv('TEST_WALLET_PRIVATE_KEY')
pay_to_address = os.getenv('NFT_WALLET_PUBLIC_KEY')

# Create account from private key
account = Account.from_key(private_key)
from_address = Web3.to_checksum_address(account.address)
pay_to_address = Web3.to_checksum_address(pay_to_address)
print(f"Payer Address: {from_address}")
print(f"Recipient Address: {pay_to_address}")

# Network and token configuration
CHAIN_ID = 11155420  # Optimism Sepolia
USDC_ADDRESS = "0x5fd84259d66Cd46123540766Be93DFE6D43130D7"  # USDC on Optimism Sepolia

# Create authorization data
valid_after = int(datetime.now().timestamp()) - 60  # Valid from 1 min ago
valid_before = int(datetime.now().timestamp()) + 3600  # Valid for 1 hour
nonce = "0x" + secrets.token_hex(32)  # Random nonce
amount = "10000"  # 0.01 USDC (6 decimals)

# EIP-712 domain for USDC
domain_data = {
    "name": "USD Coin",
    "version": "2",
    "chainId": CHAIN_ID,
    "verifyingContract": USDC_ADDRESS
}

# EIP-712 message (TransferWithAuthorization)
message_data = {
    "from": from_address,
    "to": pay_to_address,
    "value": int(amount),
    "validAfter": int(valid_after),
    "validBefore": int(valid_before),
    "nonce": bytes.fromhex(nonce[2:])  # Convert hex string to bytes
}

# EIP-712 types
types_data = {
    "EIP712Domain": [
        {"name": "name", "type": "string"},
        {"name": "version", "type": "string"},
        {"name": "chainId", "type": "uint256"},
        {"name": "verifyingContract", "type": "address"}
    ],
    "TransferWithAuthorization": [
        {"name": "from", "type": "address"},
        {"name": "to", "type": "address"},
        {"name": "value", "type": "uint256"},
        {"name": "validAfter", "type": "uint256"},
        {"name": "validBefore", "type": "uint256"},
        {"name": "nonce", "type": "bytes32"}
    ]
}

# Create the full typed data structure
typed_data = {
    "types": types_data,
    "primaryType": "TransferWithAuthorization",
    "domain": domain_data,
    "message": message_data
}

# Encode and sign
encoded_data = encode_typed_data(full_message=typed_data)
signed_message = account.sign_message(encoded_data)

# Add 0x prefix to signature (required by viem)
signature_hex = "0x" + signed_message.signature.hex() if not signed_message.signature.hex().startswith("0x") else signed_message.signature.hex()

print(f"\nSignature: {signature_hex}")
print(f"Nonce: {nonce}")
print(f"Valid After: {valid_after}")
print(f"Valid Before: {valid_before}")

Payer Address: 0x553179556FC2A39e535D65b921e01fA995E79101
Recipient Address: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C

Signature: 0xc0718d37dec5de76c99143093780eb9db2e5460d9aa8d42d3d6092e4906910627c289a530d943ac5e15f15c8e42af1982453004ffa3770c63f43f8bf7ca0651a1c
Nonce: 0xfe831fb1c698c04c32d6b31a8bfd74779d22c913465ed5252bd6471f2577e09b
Valid After: 1765915844
Valid Before: 1765919504


In [17]:
# Now create a valid x402 payment payload with our real signature
real_payment_payload = {
    "x402Version": 2,
    "resource": {
        "url": "https://api.example.com/premium-data",
        "description": "Access to premium market data",
        "mimeType": "application/json"
    },
    "accepted": {
        "scheme": "exact",
        "network": "eip155:11155420",  # Optimism Sepolia testnet
        "amount": amount,
        "asset": USDC_ADDRESS,
        "payTo": pay_to_address,
        "maxTimeoutSeconds": 60,
        "extra": {
            "name": "USDC",
            "version": "2"
        }
    },
    "payload": {
        "signature": signature_hex,
        "authorization": {
            "from": from_address,
            "to": pay_to_address,
            "value": amount,
            "validAfter": str(valid_after),
            "validBefore": str(valid_before),
            "nonce": nonce
        }
    }
}

# Create payment requirements
real_payment_requirements = {
    "scheme": "exact",
    "network": "eip155:11155420",
    "amount": amount,
    "asset": USDC_ADDRESS,
    "payTo": pay_to_address,
    "maxTimeoutSeconds": 60,
    "extra": {
        "name": "USDC",
        "version": "2"
    }
}

# Create verify request
real_verify_request = {
    "paymentPayload": real_payment_payload,
    "paymentRequirements": real_payment_requirements
}

# Send to facilitator
response = requests.post(
    f"{FACILITATOR_URL}/verify",
    headers={"Content-Type": "application/json"},
    json=real_verify_request
)

print(f"Status Code: {response.status_code}")
result = response.json()
print(json.dumps(result, indent=2))

if result.get("isValid"):
    print(f"\n✅ Payment ist GÜLTIG!")
    print(f"Payer: {result.get('payer')}")
else:
    print(f"\n❌ Payment ist UNGÜLTIG")
    print(f"Grund: {result.get('invalidReason')}")

Status Code: 200
{
  "isValid": true,
  "payer": "0x553179556FC2A39e535D65b921e01fA995E79101"
}

✅ Payment ist GÜLTIG!
Payer: 0x553179556FC2A39e535D65b921e01fA995E79101


## Example 1: Valid Payment Verification

Let's create a valid payment payload and verify it with the facilitator.

In [2]:
# Sample valid payment payload (this would be created by a client with a real wallet)
valid_payment_payload = {
    "x402Version": 2,
    "resource": {
        "url": "https://api.example.com/premium-data",
        "description": "Access to premium market data",
        "mimeType": "application/json"
    },
    "accepted": {
        "scheme": "exact",
        "network": "eip155:11155420",  # Optimism Sepolia testnet
        "amount": "10000",  # 0.01 USDC (6 decimals)
        "asset": "0x5fd84259d66Cd46123540766Be93DFE6D43130D7",  # USDC on Optimism Sepolia
        "payTo": "0x209693Bc6afc0C5328bA36FaF03C514EF312287C",
        "maxTimeoutSeconds": 60,
        "extra": {
            "name": "USDC",
            "version": "2"
        }
    },
    "payload": {
        "signature": "0x2d6a7588d6acca505cbf0d9a4a227e0c52c6c34008c8e8986a1283259764173608a2ce6496642e377d6da8dbbf5836e9bd15092f9ecab05ded3d6293af148b571c",
        "authorization": {
            "from": "0x857b06519E91e3A54538791bDbb0E22373e36b66",
            "to": "0x209693Bc6afc0C5328bA36FaF03C514EF312287C",
            "value": "10000",
            "validAfter": str(int(datetime.now().timestamp()) - 60),  # Valid from 1 min ago
            "validBefore": str(int(datetime.now().timestamp()) + 3600),  # Valid for 1 hour
            "nonce": "0xf3746613c2d920b5fdabc0856f2aeb2d4f88ee6037b8cc5d04a71a4462f13480"
        }
    }
}

# Payment requirements (what the resource server expects)
payment_requirements = {
    "scheme": "exact",
    "network": "eip155:11155420",
    "amount": "10000",
    "asset": "0x5fd84259d66Cd46123540766Be93DFE6D43130D7",
    "payTo": "0x209693Bc6afc0C5328bA36FaF03C514EF312287C",
    "maxTimeoutSeconds": 60,
    "extra": {
        "name": "USDC",
        "version": "2"
    }
}

print("Payment Payload Created")
print(f"Payer: {valid_payment_payload['payload']['authorization']['from']}")
print(f"Amount: {valid_payment_payload['payload']['authorization']['value']} (0.01 USDC)")
print(f"Network: {valid_payment_payload['accepted']['network']}")

Payment Payload Created
Payer: 0x857b06519E91e3A54538791bDbb0E22373e36b66
Amount: 10000 (0.01 USDC)
Network: eip155:11155420


### Send Verification Request

Now let's send this to the facilitator's `/verify` endpoint:

In [3]:
# Verify the payment
verify_request = {
    "paymentPayload": valid_payment_payload,
    "paymentRequirements": payment_requirements
}

try:
    response = requests.post(
        f"{FACILITATOR_URL}/verify",
        json=verify_request,
        headers={"Content-Type": "application/json"}
    )
    
    print(f"Status Code: {response.status_code}")
    print(f"\nResponse:")
    result = response.json()
    print(json.dumps(result, indent=2))
    
    if result.get("isValid"):
        print(f"\n✅ Payment is VALID!")
        print(f"Payer: {result.get('payer')}")
    else:
        print(f"\n❌ Payment is INVALID")
        print(f"Reason: {result.get('invalidReason')}")
        
except requests.exceptions.ConnectionError:
    print("⚠️  Could not connect to facilitator. Make sure it's running:")
    print("   cd scw_js/x402_facilitator")
    print("   NODE_ENV=test node x402_facilitator.js")
except Exception as e:
    print(f"Error: {e}")

Status Code: 200

Response:
{
  "isValid": false,
  "invalidReason": "invalid_exact_evm_payload_signature",
  "payer": "0x857b06519E91e3A54538791bDbb0E22373e36b66"
}

❌ Payment is INVALID
Reason: invalid_exact_evm_payload_signature


## Example 2: Invalid Payment - Insufficient Amount

Let's test what happens when the payment amount is too low:

In [ ]:
# Create payment with insufficient amount
insufficient_payment = valid_payment_payload.copy()
insufficient_payment["payload"] = insufficient_payment["payload"].copy()
insufficient_payment["payload"]["authorization"] = insufficient_payment["payload"]["authorization"].copy()
insufficient_payment["payload"]["authorization"]["value"] = "5000"  # Only 0.005 USDC

verify_request = {
    "paymentPayload": insufficient_payment,
    "paymentRequirements": payment_requirements
}

try:
    response = requests.post(
        f"{FACILITATOR_URL}/verify",
        json=verify_request,
        headers={"Content-Type": "application/json"}
    )
    
    result = response.json()
    print(json.dumps(result, indent=2))
    
    if not result.get("isValid"):
        print(f"\n❌ Expected failure: {result.get('invalidReason')}")
except Exception as e:
    print(f"Error: {e}")

## Example 3: Invalid Payment - Wrong Recipient

What if the payment is sent to the wrong address?

In [ ]:
# Create payment with wrong recipient
wrong_recipient_payment = valid_payment_payload.copy()
wrong_recipient_payment["payload"] = wrong_recipient_payment["payload"].copy()
wrong_recipient_payment["payload"]["authorization"] = wrong_recipient_payment["payload"]["authorization"].copy()
wrong_recipient_payment["payload"]["authorization"]["to"] = "0x0000000000000000000000000000000000000000"

verify_request = {
    "paymentPayload": wrong_recipient_payment,
    "paymentRequirements": payment_requirements
}

try:
    response = requests.post(
        f"{FACILITATOR_URL}/verify",
        json=verify_request,
        headers={"Content-Type": "application/json"}
    )
    
    result = response.json()
    print(json.dumps(result, indent=2))
    
    if not result.get("isValid"):
        print(f"\n❌ Expected failure: {result.get('invalidReason')}")
except Exception as e:
    print(f"Error: {e}")

## Example 4: Invalid Payment - Expired Authorization

Test an expired payment authorization:

In [ ]:
# Create payment that has expired
expired_payment = valid_payment_payload.copy()
expired_payment["payload"] = expired_payment["payload"].copy()
expired_payment["payload"]["authorization"] = expired_payment["payload"]["authorization"].copy()
expired_payment["payload"]["authorization"]["validBefore"] = "1000000000"  # Year 2001

verify_request = {
    "paymentPayload": expired_payment,
    "paymentRequirements": payment_requirements
}

try:
    response = requests.post(
        f"{FACILITATOR_URL}/verify",
        json=verify_request,
        headers={"Content-Type": "application/json"}
    )
    
    result = response.json()
    print(json.dumps(result, indent=2))
    
    if not result.get("isValid"):
        print(f"\n❌ Expected failure: {result.get('invalidReason')}")
except Exception as e:
    print(f"Error: {e}")

## Example 5: Unsupported Network

What happens if we try to use a network that's not supported?

In [ ]:
# Create payment for unsupported network
unsupported_network_payment = valid_payment_payload.copy()
unsupported_network_payment["accepted"] = unsupported_network_payment["accepted"].copy()
unsupported_network_payment["accepted"]["network"] = "eip155:1"  # Ethereum mainnet (not supported)

unsupported_requirements = payment_requirements.copy()
unsupported_requirements["network"] = "eip155:1"

verify_request = {
    "paymentPayload": unsupported_network_payment,
    "paymentRequirements": unsupported_requirements
}

try:
    response = requests.post(
        f"{FACILITATOR_URL}/verify",
        json=verify_request,
        headers={"Content-Type": "application/json"}
    )
    
    result = response.json()
    print(json.dumps(result, indent=2))
    
    if not result.get("isValid"):
        print(f"\n❌ Expected failure: {result.get('invalidReason')}")
except Exception as e:
    print(f"Error: {e}")

## Example 6: USDT0 Payment (usdt0.to)

The facilitator also supports USDT0 tokens from [usdt0.to](https://usdt0.to) on Optimism Mainnet!

USDT0 uses the same EIP-3009 `transferWithAuthorization` interface, making it compatible with x402.

In [ ]:
# Example USDT0 payment on Optimism Mainnet
usdt0_payment_payload = {
    "x402Version": 2,
    "resource": {
        "url": "https://api.example.com/premium-data",
        "description": "Access to premium market data",
        "mimeType": "application/json"
    },
    "accepted": {
        "scheme": "exact",
        "network": "eip155:10",  # Optimism Mainnet
        "amount": "10000",  # 0.01 USDT (6 decimals)
        "asset": "0x01bFF41798a0BcF287b996046Ca68b395DbC1071",  # USDT0 on Optimism
        "payTo": "0x209693Bc6afc0C5328bA36FaF03C514EF312287C",
        "maxTimeoutSeconds": 60,
        "extra": {
            "name": "Tether USD",
            "version": "1"
        }
    },
    "payload": {
        "signature": "0x...",  # Would be created with USDT0 EIP-712 domain
        "authorization": {
            "from": "0x857b06519E91e3A54538791bDbb0E22373e36b66",
            "to": "0x209693Bc6afc0C5328bA36FaF03C514EF312287C",
            "value": "10000",
            "validAfter": str(int(datetime.now().timestamp()) - 60),
            "validBefore": str(int(datetime.now().timestamp()) + 3600),
            "nonce": "0xf3746613c2d920b5fdabc0856f2aeb2d4f88ee6037b8cc5d04a71a4462f13481"
        }
    }
}

usdt0_requirements = {
    "scheme": "exact",
    "network": "eip155:10",
    "amount": "10000",
    "asset": "0x01bFF41798a0BcF287b996046Ca68b395DbC1071",
    "payTo": "0x209693Bc6afc0C5328bA36FaF03C514EF312287C",
    "maxTimeoutSeconds": 60,
    "extra": {
        "name": "Tether USD",
        "version": "1"
    }
}

print("✨ USDT0 Payment Example")
print(f"Token: USDT0 (Tether USD)")
print(f"Contract: {usdt0_payment_payload['accepted']['asset']}")
print(f"Network: Optimism Mainnet")
print(f"Amount: 0.01 USDT")
print(f"\nNote: This would work with a valid signature signed with USDT0's EIP-712 domain")

## Summary

The x402 Facilitator performs the following validations:

1. ✅ **Protocol Version** - Must be x402 v2
2. ✅ **Scheme Support** - Must be "exact"
3. ✅ **Network Support** - Must be supported Optimism network
4. ✅ **Time Window** - validAfter ≤ now < validBefore
5. ✅ **Amount Check** - Authorization value ≥ required amount
6. ✅ **Recipient Match** - Authorization.to === paymentRequirements.payTo
7. ✅ **Signature Validation** - EIP-712 signature verification
8. ✅ **Nonce Check** - Nonce not already used on-chain
9. ✅ **Balance Check** - Payer has sufficient USDC balance

### Error Codes

Common error codes you might see:

- `insufficient_funds` - Payer doesn't have enough USDC
- `invalid_exact_evm_payload_signature` - Invalid EIP-712 signature
- `invalid_exact_evm_payload_authorization_value` - Amount too low
- `invalid_exact_evm_payload_authorization_valid_before` - Expired
- `invalid_exact_evm_payload_authorization_valid_after` - Not yet valid
- `invalid_exact_evm_payload_recipient_mismatch` - Wrong recipient
- `invalid_network` - Network not supported
- `unsupported_scheme` - Scheme not supported

### Next Steps

To create real payment authorizations with valid signatures, you would need:

1. A wallet with USDC on Optimism (Sepolia for testing)
2. Sign the EIP-712 authorization with your private key
3. Use libraries like `viem` or `ethers.js` to create valid signatures

See the [Implementation Plan](../website/blog/X402_FACILITATOR_IMPLEMENTATION_PLAN.md) for more details.

## Creating a Valid Signature with Python web3

In den vorherigen Beispielen haben wir statische Signaturen verwendet, die nicht zu den dynamischen Timestamps passen. Jetzt erstellen wir eine echte, gültige Signatur mit deinem Test Wallet.